# Data Cleaning – Data Preparation Stage

This document describes the process of preparing a multilingual dataset of farmer questions for subsequent stages of word analysis, semantic dictionary construction and topic assignment.

The underlying corpus includes questions written in:
- Luganda (`lug`),
- Runyankole (`nyn`),
- Kiswahili (`swa`),
- English (`eng`).

All cleaning and categorisation steps described here are applied to the combined dataset as a whole, **without** splitting it into separate language-specific tables.

The workflow is split into two main parts:
- **Steps 1–5**: initial cleaning and simplification of the source dataset in **Power Query**.
- **Steps 6–7**: further processing in external **Python/Jupyter pipelines**, including semantic dictionary building and assigning categories back to the full dataset.

---

## 1. Downloading the Source Data

The input data was downloaded from a public URL provided in the Producers Direct project documentation.

The downloaded file was saved locally as:

**`raw_challenge_2_seasonality.csv`**

The file contains the complete set of agricultural question records, including user metadata, response information, question language (`question_language`), and topic field (`question_topic`).

---

## 2. Loading the Data into Power Query

The file `raw_challenge_2_seasonality.csv` was imported into Power Query to perform initial cleaning and data preparation.

At this stage:

- column names and data types were identified,
- key fields (including `question_language`, `question_topic`, and the question text) were verified to be available,
- the dataset was prepared for further column reduction and deduplication.

---

## 3. Removing Unnecessary Columns

In the first cleaning step, columns that were not needed for further text analysis or contained detailed user and response metadata were removed.

The Power Query operation used:

```m
= Table.RemoveColumns(#"Changed Type",
{
    "question_user_gender", "response_user_gender", "response_id",
    "response_user_id", "response_language", "response_content",
    "response_topic", "response_user_type", "response_user_status",
    "response_user_country_code", "response_user_dob",
    "response_user_created_at", "question_user_dob",
    "question_user_type", "question_user_status",
    "question_user_created_at", "response_sent",
    "question_user_id"
})
```

The purpose of this step was:

- reducing the size of the table,
- removing fields that are not used in further linguistic analysis,
- limiting the data scope to information that is actually needed in the next stages (analysis of question content, language, topic).

---

## 4. Removing Duplicates

After removing unnecessary columns, deduplication was performed at the level of the entire table:

```m
= Table.Distinct(#"Removed Columns")
```

As a result:

- duplicate question records were eliminated,
- it was ensured that word frequency analysis and subsequent statistics would not be distorted by counting the same observations multiple times.

---

## 5. Creating the `topic_category` Column (Working Thematic Categorization)

To facilitate a possible thematic breakdown in later visualizations (e.g., on a dashboard) and data exploration, a new **`topic_category`** column was created.

**Important assumption:**  
The set of categories does **not** come from the source data or from the official Producers Direct taxonomy.  
The categories were **defined independently for the purposes of this analysis** as a working, simplified thematic breakdown of agricultural questions.

The logic for creating the `topic_category` column included normalizing `question_topic` and matching it against prepared working lists of categories such as:

- `cereals`,
- `legumes`,
- `vegetables`,
- `fruits`,
- `roots_tubers`,
- `livestock`,
- `cash_crops`,
- `fodder_forage`,
- `trees`.

Questions that did not fit into any category were assigned the label `"other/unknown"`.

Below is the code (M language / Power Query) used to create this column:

```m
let
    topic =
        if [question_topic] = null then
            null
        else
            Text.Lower(Text.Trim([question_topic])),

    cereals = {
        "maize", "wheat", "barley", "rice",
        "millet", "finger-millet", "oat", "rye", "cereal"
    },

    legumes = {
        "bean", "pea", "cowpea", "pigeon-pea", "pigeon",
        "chickpea", "mung-bean", "soya", "lupin",
        "snap-pea", "snow-pea", "french-bean", "peanut"
    },

    vegetables = {
        "amaranth", "asparagus", "aubergine", "beetroot",
        "broccoli", "butternut-squash", "cabbage", "carrot",
        "cauliflower", "celery", "chard", "chilli",
        "collard-greens", "courgette", "cucumber",
        "garlic", "ginger", "greens", "kale", "leek",
        "lettuce", "okra", "onion", "parsley",
        "pumpkin", "radish", "spinach", "squash",
        "tomato", "vegetable",
        "nightshade", "black-nightshade", "african-nightshade",
        "capsicum", "corriander"
    },

    fruits = {
        "apple", "apricot", "avocado", "banana", "blackberry",
        "cranberry", "gooseberry", "grape", "guava",
        "jackfruit", "lemon", "mango", "melon", "mulberry",
        "olive", "orange", "passion-fruit", "paw-paw",
        "peach", "pear", "pineapple", "strawberry",
        "watermelon"
    },

    roots_tubers = {
        "cassava", "sweet-potato", "potato", "taro", "yam"
    },

    livestock = {
        "animal", "cattle", "goat", "sheep", "pig",
        "poultry", "chicken", "duck", "turkey",
        "rabbit", "fish", "tilapia", "ostrich",
        "guinea-fowl", "guinea-pig", "bee", "camel",
        "livestock"
    },

    cash_crops = {
        "coffee", "tea", "cocoa", "cotton", "tobacco",
        "cashew-nut", "macademia", "castor-bean",
        "flax", "pyrethrum", "miraa", "sisal",
        "rapeseed", "sesame", "sunflower", "safflower",
        "sugar-cane", "chia"
    },

    fodder_forage = {
        "boma-rhodes", "brachiaria-grass", "grass",
        "napier-grass", "sudan-grass", "setaria",
        "lucern", "desmodium", "clover",
        "leucaena", "caliandra", "purple-vetch", "vetch"
    },

    trees = {
        "acacia", "bamboo", "eucalyptus", "tree", "cyprus"
    }

in
    if topic = null or topic = "" then
        "other/unknown"
    else if List.Contains(cereals, topic) then
        "cereals"
    else if List.Contains(legumes, topic) then
        "legumes"
    else if List.Contains(vegetables, topic) then
        "vegetables"
    else if List.Contains(fruits, topic) then
        "fruits"
    else if List.Contains(roots_tubers, topic) then
        "roots_tubers"
    else if List.Contains(livestock, topic) then
        "livestock"
    else if List.Contains(cash_crops, topic) then
        "cash_crops"
    else if List.Contains(fodder_forage, topic) then
        "fodder_forage"
    else if List.Contains(trees, topic) then
        "trees"
    else
        "other/unknown"
```

At this point, the cleaned and enriched file is saved as:

**`raw_challenge_2_seasonality.csv`** (Power Query output).

This file serves as the starting point for the external semantic dictionary and classification pipeline described in the next steps.

---

## 6. Further Processing Outside Power Query – Category Dictionary Pipeline

After completing Step 5 in Power Query, the workflow does **not** continue with additional Power Query transformations.

Instead, the next part of the process is carried out entirely outside Power Query, using a dedicated Python/Jupyter pipeline documented in:

[**agricultural_semantic_dictionary_builder.ipynb**](agricultural_semantic_dictionary_builder.ipynb)

This notebook contains a detailed description of the full semantic dictionary–building workflow, including:

- extraction of unique words from the cleaned dataset `raw_challenge_2_seasonality.csv`,  
- token normalisation and filtering,  
- generation of word batches based on frequency,  
- classification of terms into agricultural categories using a GPT-based model,  
- consolidation of JSON batch outputs into a single dictionary file.

The result of this pipeline is the consolidated dictionary:

**`words_categories_merged.json`**

which contains the final category–to–word mappings in the form:

```json
{
  "planting_growing": ["..."],
  "livestock": ["..."],
  "pests_disease": ["..."],
  "timing_harvest": ["..."],
  "weather": ["..."],
  "market_price": ["..."]
}
```

This dictionary serves as the reference for assigning categories to full question texts in the final step.

---

## 7. Final Step – Assigning Categories to the Full RAW Dataset

The final stage of the workflow applies the consolidated semantic dictionary to the entire dataset of farmer questions contained in:

**`raw_challenge_2_seasonality.csv`**

This step is implemented in a standalone Python script:

**`merge_raw_with_categories.py`**

The script performs the following operations:

### 7.1. Load the classification results

A CSV file produced earlier in the pipeline (`topics_with_categories_one_col.csv`) is read into memory and converted into a lookup dictionary:

`question_content → category`

Each entry corresponds to the category assigned to that question’s text based on the semantic dictionary and GPT-based classification logic.

### 7.2. Stream the full RAW dataset in chunks

The file:

**`raw_challenge_2_seasonality.csv`**

is processed in chunks of 100,000 rows to ensure efficient memory usage and to avoid loading the entire dataset into memory at once.

### 7.3. Assign one category per question

For each row:

- If the question text (`question_content`) exists in the classification lookup → its corresponding category is assigned.  
- If the question text is not found in the lookup → the category `"other"` is assigned.

This guarantees **exactly one category** for every question in the dataset.

### 7.4. Export the final enriched dataset

The output is saved as:

**`processed_with_categories.csv`**

This file contains:

- all columns carried over from the cleaned raw dataset,  
- plus one new column:

`category`

which holds the final category assigned to each question based on the semantic dictionary.

---

## Final Output Summary

The complete workflow:

1. Cleans and simplifies the original dataset in Power Query (Steps 1–5),  
2. Builds a semantic category dictionary in a dedicated Jupyter pipeline (`agricultural_semantic_dictionary_builder.ipynb`),  
3. Applies that dictionary to every question using a Python script (`merge_raw_with_categories.py`) to produce a final, categorised dataset.

The main final output of this end-to-end process is:

### **`processed_with_categories.csv`**

A fully processed, classified dataset containing:

- original farmer questions,  
- relevant metadata from the cleaned raw dataset,  
- a final category label (`category`) derived from the semantic dictionary and GPT-based classification.

This dataset is the primary input for subsequent analysis, visualisation, and modelling (e.g., dashboards, exploratory analysis of topics, or further NLP experiments).

